In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
V0=0.02
θ=0.02
λ=0.3
ν=0.3
ρ=-0.7
H=0.1
S0=100

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pyfeng as pf
import LiftedHeston as LF
import matplotlib.pyplot as plt
from matplotlib import rcParams, cycler
import numpy as np
import pandas as pd

In [10]:
import xlrd
data=xlrd.open_workbook("rough heston impvol.xls")
data_log_moneyness=data.sheet_by_name("log_moneyness")
data_impvol=data.sheet_by_name("impvol")

In [ ]:
n=[10,20]
n=np.asarray(n)
r=1+10*pow(n,-0.9)
maturities=[7/365,1/12,1/6,1/4,1/2,3/4,1,1.5,2]
titles=['1 week','1 month','2 months','3 months','6 months','9 months','1 year','1.5 years','2 years']
colors=['b','#00FFFF','#00FF00','#FFFF00','#FF8C00']
MSE=np.zeros(shape=(9,5))

i_row=6 #change here
log_moneyness=np.asarray(data_log_moneyness.row_values(i_row))
impvol_rough=np.asarray(data_impvol.row_values(i_row))
strikes=S0*np.exp(log_moneyness)

df=pd.DataFrame(log_moneyness)                                                                                                                                                                                                                                                                                                                                               
for i_n in range(2):
    LH=LF.LiftedHeston(vov=ν, v0=V0, rho=ρ, theta=θ, lamda=λ, H=H,n=n[i_n],rn=r[i_n])
    prices_lifted=LH.price(strike=strikes, spot=S0, texp=maturities[i_row], cp=1)
    BSM_model=pf.Bsm(0.3)
    impvol_lifted=BSM_model._impvol_newton(prices_lifted,strikes,S0,maturities[i_row]) 
    plt.plot(log_moneyness,impvol_lifted,color=colors[i_n])
    MSE[i_row,i_n]=np.mean(np.power(impvol_lifted-impvol_rough,2))
    df["n="+str(n[i_n])]=impvol_lifted
df.to_excel("add_maturity"+str(maturities[i_row])+".xlsx")
plt.plot(log_moneyness,impvol_rough,color='r',linewidth=2)
plt.legend(['n=10','n=20'])
plt.xlabel('log moneyness',fontsize=12)
plt.ylabel('implied volatility',fontsize=12)
plt.title('T='+titles[i_row],fontsize=12)
plt.show()
plt.savefig("add_maturity"+str(maturities[i_row])+".png")